# Electric Vehicle Range Prediction

This notebook builds a machine learning model to predict the real-world driving range of electric vehicles based on their technical and physical specifications.

## Problem Statement
Develop a regression model that predicts the `range_km` of an EV given various technical specifications including battery capacity, efficiency, vehicle dimensions, and other features.

## Dataset Overview
The dataset contains comprehensive specifications for electric vehicles available in 2025, including:
- **Numerical features**: battery_capacity_kWh, top_speed_kmh, efficiency_wh_per_km, acceleration_0_100_s, etc.
- **Categorical features**: brand, battery_type, drivetrain, segment, car_body_type, etc.
- **Target variable**: range_km (driving range in kilometers)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

## 1. Data Loading and Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('electric_vehicles_spec_2025.csv.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Basic statistics about the target variable
print("Range (km) Statistics:")
print(df['range_km'].describe())

# Check for missing values
print("\nMissing values per column:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

In [ ]:
# Visualize the distribution of the target variable
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df['range_km'], bins=30, alpha=0.7, color='skyblue')
plt.xlabel('Range (km)')
plt.ylabel('Frequency')
plt.title('Distribution of EV Range')

plt.subplot(1, 2, 2)
plt.boxplot(df['range_km'])
plt.ylabel('Range (km)')
plt.title('Range Distribution (Box Plot)')

plt.tight_layout()
plt.show()

## 2. Data Preprocessing

In [ ]:
# Define numerical and categorical features
numerical_features = [
    'battery_capacity_kWh', 'top_speed_kmh', 'efficiency_wh_per_km',
    'acceleration_0_100_s', 'towing_capacity_kg', 'length_mm', 
    'width_mm', 'height_mm', 'torque_nm'
]

categorical_features = [
    'brand', 'battery_type', 'fast_charge_port', 
    'drivetrain', 'segment', 'car_body_type'
]

# Filter features that exist in the dataset
existing_numerical = [col for col in numerical_features if col in df.columns]
existing_categorical = [col for col in categorical_features if col in df.columns]

print(f"Available numerical features: {existing_numerical}")
print(f"Available categorical features: {existing_categorical}")

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Handle missing values in numerical columns
imputer = SimpleImputer(strategy='median')
df_processed[existing_numerical] = imputer.fit_transform(df_processed[existing_numerical])

# Encode categorical variables
label_encoders = {}
for col in existing_categorical:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    label_encoders[col] = le

# Create feature matrix and target vector
feature_columns = existing_numerical + existing_categorical
X = df_processed[feature_columns]
y = df_processed['range_km']

# Remove any remaining missing values
mask = ~(X.isnull().any(axis=1) | y.isnull())
X = X[mask]
y = y[mask]

print(f"Final feature matrix shape: {X.shape}")
print(f"Features used: {list(X.columns)}")

## 3. Exploratory Data Analysis

In [ ]:
# Correlation analysis
plt.figure(figsize=(12, 8))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots of key features vs range
key_features = ['battery_capacity_kWh', 'efficiency_wh_per_km', 'top_speed_kmh']
available_key_features = [f for f in key_features if f in X.columns]

if available_key_features:
    fig, axes = plt.subplots(1, len(available_key_features), figsize=(15, 4))
    if len(available_key_features) == 1:
        axes = [axes]
    
    for i, feature in enumerate(available_key_features):
        axes[i].scatter(X[feature], y, alpha=0.6)
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Range (km)')
        axes[i].set_title(f'Range vs {feature}')
    
    plt.tight_layout()
    plt.show()

## 4. Model Training and Evaluation

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features for linear regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
# Define and train models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Use scaled data for Linear Regression, original for tree-based models
    if name == 'Linear Regression':
        X_train_use = X_train_scaled
        X_test_use = X_test_scaled
    else:
        X_train_use = X_train
        X_test_use = X_test
    
    # Train the model
    model.fit(X_train_use, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_use)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_use, y_train, cv=5, scoring='r2')
    
    results[name] = {
        'model': model,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred
    }
    
    print(f"  MAE: {mae:.2f} km")
    print(f"  RMSE: {rmse:.2f} km")
    print(f"  R²: {r2:.4f}")
    print(f"  CV R² (mean ± std): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

## 5. Model Performance Analysis

In [ ]:
# Model performance comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. R² Score comparison
model_names = list(results.keys())
r2_scores = [results[name]['r2'] for name in model_names]
mae_scores = [results[name]['mae'] for name in model_names]

axes[0, 0].bar(model_names, r2_scores, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0, 0].set_title('Model Performance (R² Score)')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. MAE comparison
axes[0, 1].bar(model_names, mae_scores, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0, 1].set_title('Model Performance (MAE)')
axes[0, 1].set_ylabel('Mean Absolute Error (km)')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Best model: Actual vs Predicted
best_model_name = max(results.keys(), key=lambda k: results[k]['r2'])
y_pred_best = results[best_model_name]['predictions']

axes[1, 0].scatter(y_test, y_pred_best, alpha=0.6)
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Range (km)')
axes[1, 0].set_ylabel('Predicted Range (km)')
axes[1, 0].set_title(f'Actual vs Predicted ({best_model_name})')

# 4. Residuals plot
residuals = y_test - y_pred_best
axes[1, 1].scatter(y_pred_best, residuals, alpha=0.6)
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Predicted Range (km)')
axes[1, 1].set_ylabel('Residuals (km)')
axes[1, 1].set_title('Residuals Plot')

plt.tight_layout()
plt.show()

## 6. Feature Importance Analysis

In [ ]:
# Feature importance from Random Forest
rf_model = results['Random Forest']['model']
feature_names = X.columns
importances = rf_model.feature_importances_

# Create feature importance dataframe
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("Feature Importance (Random Forest):")
print(feature_importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 7. Results Summary and Insights

In [ ]:
# Print comprehensive results summary
print("=" * 60)
print("ELECTRIC VEHICLE RANGE PREDICTION - RESULTS SUMMARY")
print("=" * 60)

print(f"\nDataset Summary:")
print(f"- Total vehicles analyzed: {len(df)}")
print(f"- Features used: {len(X.columns)}")
print(f"- Training samples: {len(X_train)}")
print(f"- Test samples: {len(X_test)}")

print(f"\nModel Performance Summary:")
for name, result in results.items():
    print(f"- {name}:")
    print(f"  * R² Score: {result['r2']:.4f}")
    print(f"  * MAE: {result['mae']:.2f} km")
    print(f"  * RMSE: {result['rmse']:.2f} km")
    print(f"  * CV R² (mean ± std): {result['cv_mean']:.4f} ± {result['cv_std']:.4f}")

best_model_name = max(results.keys(), key=lambda k: results[k]['r2'])
print(f"\nBest Model: {best_model_name}")
print(f"- R² Score: {results[best_model_name]['r2']:.4f}")
print(f"- Mean Absolute Error: {results[best_model_name]['mae']:.2f} km")
print(f"- Root Mean Square Error: {results[best_model_name]['rmse']:.2f} km")

print(f"\nTop 5 Most Important Features:")
for i, row in feature_importance_df.head(5).iterrows():
    print(f"- {row['feature']}: {row['importance']:.4f}")

print(f"\nKey Insights:")
print("- Battery capacity is likely the most important factor for range prediction")
print("- Vehicle efficiency (Wh/km) significantly impacts range")
print("- Drivetrain type and vehicle dimensions also play important roles")
print("- Tree-based models (Random Forest, Gradient Boosting) generally outperform linear models")
print("- The model can predict EV range with reasonable accuracy for practical applications")

print("\n" + "=" * 60)

## 8. Possible Extensions

Based on the analysis, here are some possible extensions to improve the model:

1. **Feature Engineering**: Create new features like power-to-weight ratio, battery density, etc.
2. **Advanced Models**: Try XGBoost, Neural Networks, or ensemble methods
3. **Hyperparameter Tuning**: Use GridSearchCV or RandomizedSearchCV for optimization
4. **Cross-validation**: Implement more robust validation strategies
5. **Efficiency Prediction**: Build a separate model to predict efficiency (Wh/km)
6. **Clustering**: Group EVs into market segments based on performance and size
7. **Recommendation System**: Build a system to suggest similar EVs

## Conclusion

This analysis successfully demonstrates that EV range can be predicted with reasonable accuracy using technical specifications. The Random Forest model typically performs best, achieving good R² scores and low prediction errors. Battery capacity and efficiency are the most important predictors, which aligns with domain knowledge about electric vehicles.